## 사용자 데이터 수집
---
*API 이용 제한 & 수집 결과 확인을 위하여 티어별로 수집 진행

### 아이언

In [2]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-8953b1ed-694b-4425-a8e0-624022d081bd'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):
    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_iron_test.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/iron_match.csv', encoding = 'cp949')
    df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()
        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1
main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 35000/35000 [13:31:39<00:00,  1.39s/it]


In [1]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-d551f477-7453-49a0-9e13-2a7d45d406aa'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_iron_test_data.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/iron_match.csv', encoding = 'cp949')
    #df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list_iron.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 43168/43168 [16:46:49<00:00,  1.40s/it]


In [ ]:
final_result

### 브론즈

In [1]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-c0ee6f4a-2a38-45dd-883b-3a2f7e27782a'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_bronze_test.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/bronze_match.csv', encoding = 'cp949')
    df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list_bronze.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 35000/35000 [13:34:47<00:00,  1.40s/it]


In [2]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-75713e2b-b873-48a5-88e8-97fac9571477'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_bronze_test_data.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/bronze_match.csv', encoding = 'cp949')
    #df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list_bronze.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 46555/46555 [18:03:35<00:00,  1.40s/it]


### 실버

In [1]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-5a06a9b9-7fc5-4f8e-91e0-6c50a7fcd766'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_silver_test.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/silver_match.csv', encoding = 'cp949')
    #df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list_silver.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 46870/46870 [18:10:36<00:00,  1.40s/it]


### 골드

In [2]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-3305089f-3d5f-4a60-a29b-749e6ae334fe'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_gold_test.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/gold_match.csv', encoding = 'cp949')
    #df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list_gold.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 46769/46769 [18:05:43<00:00,  1.39s/it]


In [1]:
print("2023-05-07")

2023-05-07


In [1]:
print("2023-05-07")

2023-05-07


### 플래티넘

In [2]:
import pandas as pd
import pprint
import requests
import time
import csv
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-4b39cab9-414a-42f4-8f86-ad529caaa17c'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'match_platinum_test.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('match data pre/platinum_match.csv', encoding = 'cp949')
    #df = df.head(35000)
    read_data = df['matchID']
    for i in tqdm(range(k, len(read_data))):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list_platinum.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        #print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

100%|█████████████████████████████████████████████████████████████████████████| 42862/42862 [16:36:49<00:00,  1.40s/it]
